In [9]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import OneHotEncoder
import joblib

ruta = r"C:\Users\uzgre\Codes\Python\Datathon\Reto Oxxo"
archivo = os.path.join(ruta, "train_w_label.csv")

# === Lectura del archivo ===
with open(archivo, 'r', encoding='utf-8', errors='replace') as f:
    Data = pd.read_csv(f)

if 'Unnamed: 0' in Data.columns:
    Data = Data.drop(columns=['Unnamed: 0'])

Data = Data.drop(columns=['TIENDA_ID', 'DATASET'])

# === MTS2VENTAS_NUM ===
Data['MTS2VENTAS_NUM'] = Data['MTS2VENTAS_NUM'].replace(0.0, np.nan)
media_global = Data['MTS2VENTAS_NUM'].mean()
Data['MTS2VENTAS_NUM'] = Data.groupby('PLAZA_CVE')['MTS2VENTAS_NUM'].transform(lambda x: x.fillna(x.mean()))
Data['MTS2VENTAS_NUM'] = Data['MTS2VENTAS_NUM'].fillna(media_global)

# === NIVEL SOCIOECONÓMICO ===
orden_nivel = {'A': 1, 'AB': 2, 'B': 3, 'BC': 4, 'C': 5, 'CD': 6, 'D': 7}
Data['NIVELSOCIOECONOMICO_DES'] = Data['NIVELSOCIOECONOMICO_DES'].astype(str).map(orden_nivel)
Data['NIVELSOCIOECONOMICO_DES'] = Data['NIVELSOCIOECONOMICO_DES'].fillna(4)

# === Codificación categórica con OneHotEncoder ===
cat_vars = ['ENTORNO_DES', 'SEGMENTO_MAESTRO_DESC', 'LID_UBICACION_TIENDA']
Data['SEGMENTO_MAESTRO_DESC'] = Data['SEGMENTO_MAESTRO_DESC'].fillna("Missing")

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded = encoder.fit_transform(Data[cat_vars])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(cat_vars), index=Data.index)

# Guardamos el encoder
joblib.dump(encoder, os.path.join(ruta, 'onehot_encoder.pkl'))

# Integración final
Data = pd.concat([Data.drop(columns=cat_vars), encoded_df], axis=1)
Data = Data.drop_duplicates()
Data.to_csv(os.path.join(ruta, "Dataset_Train_limpio.csv"), index=False)

print("✅ Dataset de entrenamiento procesado y encoder guardado.")



✅ Dataset de entrenamiento procesado y encoder guardado.


In [10]:
# === Lectura test ===
ruta = r"C:\Users\uzgre\Codes\Python\Datathon\Reto Oxxo"
archivo = os.path.join(ruta, "test_w_label.csv")

# === Lectura del archivo ===
with open(archivo, 'r', encoding='utf-8', errors='replace') as f:
    Data = pd.read_csv(f)

if 'Unnamed: 0' in Data.columns:
    Data = Data.drop(columns=['Unnamed: 0'])

Data = Data.drop(columns=['TIENDA_ID', 'DATASET'])


# === MTS2VENTAS_NUM ===
Data['MTS2VENTAS_NUM'] = Data['MTS2VENTAS_NUM'].replace(0.0, np.nan)
media_global = Data['MTS2VENTAS_NUM'].mean()
Data['MTS2VENTAS_NUM'] = Data.groupby('PLAZA_CVE')['MTS2VENTAS_NUM'].transform(lambda x: x.fillna(x.mean()))
Data['MTS2VENTAS_NUM'] = Data['MTS2VENTAS_NUM'].fillna(media_global)

# === NIVEL SOCIOECONÓMICO ===
orden_nivel = {'A': 1, 'AB': 2, 'B': 3, 'BC': 4, 'C': 5, 'CD': 6, 'D': 7}
Data['NIVELSOCIOECONOMICO_DES'] = Data['NIVELSOCIOECONOMICO_DES'].astype(str).map(orden_nivel)
Data['NIVELSOCIOECONOMICO_DES'] = Data['NIVELSOCIOECONOMICO_DES'].fillna(4)

# === Codificación categórica con encoder guardado ===
from joblib import load
encoder = load(os.path.join(ruta, 'onehot_encoder.pkl'))

cat_vars = ['ENTORNO_DES', 'SEGMENTO_MAESTRO_DESC', 'LID_UBICACION_TIENDA']
Data['SEGMENTO_MAESTRO_DESC'] = Data['SEGMENTO_MAESTRO_DESC'].fillna("Missing")

encoded = encoder.transform(Data[cat_vars])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(cat_vars), index=Data.index)

# Integración final
Data = pd.concat([Data.drop(columns=cat_vars), encoded_df], axis=1)
Data = Data.drop_duplicates()
Data.to_csv(os.path.join(ruta, "Dataset_Test_limpio.csv"), index=False)

print("✅ Dataset de prueba procesado usando el mismo encoder.")


✅ Dataset de prueba procesado usando el mismo encoder.
